In [1]:
import pandas as pd
data = pd.read_csv('training_data.csv')
data.describe()
test_well = data[data['Well Name'] == 'SHANKLE']
data = data[data['Well Name'] != 'SHANKLE']

In [2]:
features = ['GR', 'ILD_log10', 'DeltaPHI','PHIND','PE','NM_M', 'RELPOS']
feature_vectors = data[features]
facies_labels = data['Facies']
facies_labels.describe()

count    2783.000000
mean        4.558390
std         2.515249
min         1.000000
25%         2.000000
50%         4.000000
75%         7.000000
max         9.000000
Name: Facies, dtype: float64

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_vectors)
scaled_features = scaler.transform(feature_vectors) #ndarray now. 

In [4]:
from sklearn.cross_validation import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(scaled_features, facies_labels,test_size=0.05, random_state=42)
X_train

C:\Users\jchen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


array([[-0.87705417,  0.33637903, -0.18928865, ...,  1.44783881,
         0.95329909, -1.63528358],
       [-0.19279901,  0.32811041, -0.25469982, ...,  0.61894043,
         0.95329909,  1.12000618],
       [ 0.64782298, -1.20352041,  0.12785645, ..., -0.88896106,
        -1.04898873, -1.26304115],
       ...,
       [ 0.2221964 ,  0.04857211,  0.98018391, ..., -0.74689616,
        -1.04898873,  0.94954002],
       [ 0.53236719, -0.39031519, -0.20911021, ..., -0.96726455,
        -1.04898873,  0.72341145],
       [ 0.34268302, -0.51115421,  1.13875646, ..., -0.78940377,
        -1.04898873, -1.33261918]])

In [5]:
from sklearn.metrics import classification_report
target_names = ['SS', 'CSiS', 'FSiS', 'SiSh','MS', 'WS', 'D','PS', 'BS']

In [6]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=4,min_samples_leaf=10)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [7]:
y_cv_pred = clf.predict(X_cv) 
print(classification_report(y_cv, y_cv_pred,target_names=target_names))

             precision    recall  f1-score   support

         SS       1.00      0.50      0.67        16
       CSiS       0.61      0.86      0.71        29
       FSiS       0.55      0.43      0.48        14
       SiSh       0.25      0.29      0.27         7
         MS       0.00      0.00      0.00        18
         WS       0.44      0.83      0.58        23
          D       0.00      0.00      0.00         3
         PS       0.62      0.57      0.59        23
         BS       0.50      0.57      0.53         7

avg / total       0.51      0.55      0.50       140



C:\Users\jchen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
y_test = test_well['Facies']

In [9]:
well_features = test_well.drop(['Facies','Formation','Well Name','Depth'],axis=1)
X_test = scaler.transform(well_features)
y_pred = clf.predict(X_test)
test_well['Prediction'] = y_pred

In [10]:

print(classification_report(y_test, y_pred,target_names=target_names))

             precision    recall  f1-score   support

         SS       0.43      0.07      0.12        89
       CSiS       0.31      0.81      0.45        89
       FSiS       0.82      0.36      0.50       117
       SiSh       0.06      0.14      0.09         7
         MS       0.00      0.00      0.00        19
         WS       0.56      0.80      0.66        71
          D       1.00      0.24      0.38        17
         PS       0.52      0.40      0.45        40

avg / total       0.53      0.44      0.40       449



C:\Users\jchen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 9
  .format(len(labels), len(target_names))
C:\Users\jchen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
